In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup

import nltk
import datetime
nltk.data.path.append(r"/Users/seb/Workspace/Dev/Formation-OC/LIBRAIRIES/nltk_data")

import nltk
import datetime
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split

In [2]:
file1 = 'data/QueryResults3_new.csv'
df1 = pd.read_csv(file1,index_col=['Id'])

#import mes_fonctions_final as stt
import mes_fonctions_final2 as stt
import os

StopWord = stt.load_stop_word()

In [3]:
import imp
imp.reload(stt)

<module 'mes_fonctions_final2' from '/Users/seb/Workspace/Dev/Formation-OC/Formation-Data-Scientist/Projet6-Categoriser_Automatiquement_des_Questions/NoteBooks/PROJET_A_LIVRER/mes_fonctions_final2.py'>

## Rappel des Fonctions

In [4]:
from nltk.stem.snowball import EnglishStemmer
stemmer = EnglishStemmer()

def preProcess_remove_Code(html_in):
    # Chargement du module BeautifulSoup pour le parsing des données HTML
    from bs4 import BeautifulSoup
    html_in_soup = BeautifulSoup(html_in, 'html.parser')

    # Suppression des blocs de Code
    for h_code in html_in_soup.find_all('code'):
        h_code.decompose()

    return html_in_soup.get_text().lower()


def preProcess_keep_Code(html_in):
    # Chargement du module BeautifulSoup pour le parsing des données HTML
    from bs4 import BeautifulSoup
    html_in_soup = BeautifulSoup(html_in, 'html.parser')

    return html_in_soup.get_text().lower()

# Le préprocessing :
#   - nettoie le html
#   - supprime les blocs de Code
#   - ne prend que la partie textuelle du html
#   - renvoie ce texte en minuscule


def preProcess(html_in):
    # Chargement du module BeautifulSoup pour le parsing des données HTML
    from bs4 import BeautifulSoup
    html_in_soup = BeautifulSoup(html_in, 'html.parser')

    # Suppression des blocs de Code
    for h_code in html_in_soup.find_all('code'):
        h_code.decompose()

    return html_in_soup.get_text().lower()


# Réalisation du stemming (on coupe les racines)
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed


# La tokenisation :


def tokenize(text):
    m_token_pattern = r"((?:(?:(?:[0-9a-zA-Z])\.){2,}[a-zA-Z])" +\
        "|(?:(?:[0-9a-zA-Z]){2,}\.(?:[0-9a-zA-Z]){2,}" +\
        "|(?:\.(?:[0-9a-zA-Z]){2,}))" +\
        "|[0-9a-zA-Z-\-\+\#]{2,}|w+)"

    from nltk.tokenize import RegexpTokenizer
    # Nous allons utiliser le pattern pour identifier les mots
    tokenizer = RegexpTokenizer(m_token_pattern)

    # Nous lançons la séparation des mots
    tokens = tokenizer.tokenize(text)

    # On fait appel au stemming pour rapprocher les mots de même racine
    stems = stem_tokens(tokens, stemmer)

    # Etape de nettoyage des valeurs :
    # Nous allons supprimer les nombres sans texte,
    def suppress_nb(x):
        import re
        if x is None:
            return None
        pattern = r'(^[\#\-\+]*[0-9]*$|' + '^[0-9]*[\#\-\+]*$|'+'^[0-9]*[\#\-\+]?[0-9]*$|'+'^[0-9\#\-\+][a-z]$|'+'^[a-z][0-9\#\-\+]$|'+'^[0-9]*\.[0-9]*$)'
        if not(re.match(pattern, x)):
            return x

    def nettoie_points(x):
        import re
        if x is None:
            return None

        if (re.match(r'(^[\.\-\#][a-z]*$)', x)):
            return ''.join(list(x)[1:])
        else:
            return x

    # Nous filtrons les nombres seuls
    stems = list(filter(lambda x: suppress_nb(x), stems))
    stems = [nettoie_points(x) for x in stems]
    return stems

# Fabrication des données

In [7]:
Y_tags, Y_tags_voc = stt.genere_target_dummy_and_vocabulary(data=df1, min_df=300)
Tags_freq = stt.genere_df_target_tags_freq(data=df1, min_df=300)

/!\ Ajout des features en cours


In [8]:
# On sépare les données d'entrainement de celles de tests
debut = datetime.datetime.now()
X = df1['Body']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y_tags, test_size=0.2, random_state=42 )
fin = datetime.datetime.now()
print("Fin Train_Split : {}".format(fin-debut))

Fin Train_Split : 0:00:00.055921


# Fonctions supplémentaires
Etant donné que les tags sont multi valués, nous allons 
- créer des groupes de tags
- limiter le nombre de tags pour les groupes représentant une faible fréquence
- nous mettons un seuil de couverture par les groupes créés
- nous encodons les multiples valeurs des tags
- nous pouvons aussi décoder ceci 

In [33]:
def fabrique_nom(row):
        msg = ''
        for i in row.index.tolist():
            msg+=str(row[i])
        return msg
    
def genere_encodage(seuil=0.8, Y_tags=Y_tags):
    global dict_conversion
    global conversion
    global m_colonne
 
    def fabrique_nom(row):
        msg = ''
        for i in row.index.tolist():
            msg+=str(row[i])
        return msg

    def verif_seuil(row, seuil, remplacement=-1):
        if row['CumSum_Norm']<seuil:
            return row.name 
        else:
            return remplacement

    tmp1 = Y_tags.to_dense()
    m_colonne = tmp1.columns.tolist()
    tmp1 = tmp1.groupby(by=m_colonne).size().reset_index(name='Freq')

    tmp1.sort_values(by=['Freq'], ascending=False, inplace=True)

    name1 = tmp1[m_colonne].apply(lambda row: fabrique_nom(row=row), axis=1)
    from sklearn import preprocessing
    le = preprocessing.LabelEncoder()
    tmp2 = le.fit_transform(name1)

    tmp1['Name'] = name1
    tmp1['Value1'] = tmp2

    # On ajoute la somme cumulée normalisée
    tmp1['CumSum_Norm'] = tmp1['Freq'].cumsum()/(tmp1['Freq'].sum())


    ## On transforme l'id selon le seuil de Fréquence du groupe
    tmp = tmp1.apply(lambda row: verif_seuil(row, seuil=seuil), axis=1)

    # On encode les noms 
    le2 = preprocessing.LabelEncoder()
    tmp2 = le2.fit_transform(tmp)

    tmp1['index_New'] = tmp2

    dict_conversion = tmp1
    ########del tmp1


    ## GESTION DE LA TRANSFORMATION

    # On fait la conversion entre les tags en INPUT et la valeur encodée
    # Le dictionnaire est le suivant

    conversion = {'INPUT': dict_conversion.Name}
    #####conversion['OUTPUT'] = np.array(u2.loc[le.transform(conversion['INPUT'])]['index_New'])
    conversion['OUTPUT'] = dict_conversion.index_New

    tmp_df = pd.DataFrame(conversion)
    conversion = tmp_df[np.logical_not(tmp_df.duplicated(subset=['OUTPUT'], keep='first'))]

    
def tags_binaire_2_value(value_in):
    v1 = conversion.set_index(keys=['INPUT'])
    return v1.loc[value_in, 'OUTPUT']

def value_to_tagsBinaires(value_in):
    v1 = conversion.set_index(keys=['OUTPUT'])
    return v1.loc[value_in, 'INPUT']

def value_to_matrix_tags(value_in):
    '''value_to_matrix_tags([0, 1])'''
    binary_value = value_to_tagsBinaires(value_in)
    v2 = dict_conversion.set_index(keys=['Name'])
    return pd.DataFrame(v2.loc[binary_value][m_colonne])

def encode_Y(data_in):
    u = data_in.apply(lambda row: fabrique_nom(row=row), axis=1)
    valeur_remplacement = dict_conversion.head(1)['index_New'][0]
    return pd.Series(data=tags_binaire_2_value(u).fillna(value=valeur_remplacement).values, 
                     index=data_in.index)

decode_Y_pred = value_to_matrix_tags

## Fonction de mesure de performance

In [10]:
def mesure_performance(y_pred_encoded, Y_test):
    m_Y_PRED = decode_Y_pred(y_pred_encoded)
    m_Y_PRED.index = Y_test.index
    m_res = Y_test==m_Y_PRED
    print('\tPERFORMANCE MODELE : {}  -- soit {} %'.format(
        np.mean(np.sum(m_res, axis=1)), round(np.mean(np.sum(m_res, axis=1))/58.0, 4)))

# Fabrication du Bag Of Words

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer

vect_tfidf = TfidfVectorizer(
    analyzer="word",
    min_df=100,
    max_df=0.2,
    tokenizer=tokenize,
    preprocessor=preProcess_keep_Code,
    stop_words=StopWord,        # Pas de stopWord car nous les cherchons
    ngram_range=(1, 6)
)

debut = datetime.datetime.now()
X_train2 = vect_tfidf.fit_transform(X_train)
X_test2 = vect_tfidf.transform(X_test)
fin = datetime.datetime.now()
print("Fin TfidfVectorizer : {}".format(fin-debut))


Fin TfidfVectorizer : 0:03:13.854340


# Conservation de copies des données et des modèles

# Entrainement d'un lda

## Sur le body

In [14]:
from sklearn.decomposition import LatentDirichletAllocation
no_topics = 20

# Run LDA
debut = datetime.datetime.now()
lda = LatentDirichletAllocation(
    n_components=no_topics, max_iter=5,
    learning_method='online', learning_offset=50.,random_state=0).fit(X_train2)

fin = datetime.datetime.now()
print('Tps de lda {}'.format(fin-debut))

Tps de lda 0:00:31.457904


In [20]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print("\t"," ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 10
display_topics(lda, vect_tfidf.get_feature_names(), no_top_words)

Topic 0:
	 date td tabl tr month th id day td td row
Topic 1:
	 python lib line usr import site-packag modul instal python3 bin
Topic 2:
	 public string void privat overrid public void int public class overrid public intent
Topic 3:
	 map option select countri option option marker lat locat latitud longitud
Topic 4:
	 sub dim sheet end cell excel rang worksheet vba row
Topic 5:
	 angular compon import observ ts export export class subscrib ionic pipe
Topic 6:
	 self func branch def let anim init nil tableview git
Topic 7:
	 imag color width height img style posit div left chart
Topic 8:
	 pdf notif chrome selenium firefox browser mobil webdriv paper safari
Topic 9:
	 system.out println system.out println public string static int void public static static void
Topic 10:
	 ifram keyboard timer focus delay mous true return tap creation id return
Topic 11:
	 div button div class div div click span form li input id
Topic 12:
	 app server project applic instal build version test window conne

> On constate que les topics sont bien marqués
- le topic 1 est du python
- le topic 18 de l'android
- le topic 0 du html etc ...

# Modélisation par NMF

## Pour le Body

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF

# On fait la transfo Tf-Idf
from sklearn.feature_extraction.text import TfidfTransformer
debut = datetime.datetime.now()

# Attention nous ne voulons pas la forme TFiDF donc use_idf=False
#tf = TfidfTransformer(use_idf=True)
#train_data_features_fitted_body = tf.fit_transform(X_body_stt)

#tfidf_feature_names_body = X_body_stt.columns.tolist()
no_topics = 20

debut = datetime.datetime.now()

# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(
    X_train2)

fin = datetime.datetime.now()
print('Tps de NMF BODY {}'.format(fin-debut))

Tps de NMF BODY 0:00:09.985290


In [22]:
no_top_words = 10
display_topics(nmf, vect_tfidf.get_feature_names(), no_top_words)

Topic 0:
	 instal project version build app node modul packag command applic
Topic 1:
	 div div class div div span div div class div div div li href width style
Topic 2:
	 string public void privat public void public class static overrid null public string
Topic 3:
	 id id id select queri id name product order group null item
Topic 4:
	 print line number output end def rang input result string
Topic 5:
	 android layout android layout app parent layout height layout width android layout width android layout height activ
Topic 6:
	 imag img upload width imag descript height size enter imag descript enter imag descript
Topic 7:
	 server connect client port databas host password sql socket send
Topic 8:
	 import compon const modul export react angular prop path render
Topic 9:
	 button click page view text event style show html item
Topic 10:
	 date month format day date date year datetim date format mm start date
Topic 11:
	 int std int int includ char printf cout i++ main void
Topic 12:


# Modélisation LDA

## Etapes
1. Nous fabriquons les noms des Groupes sur chaque POST
2. Nous comptons la fréquence de chaque groupe
3. Nous identifions le nombre 'négligeables' de groupes
4. Nous pouvons lancer la modélisation lda qui va générer le nombre de topics correspondant au nombre de groupe
5. On modélise l'appartenance un topic vers un groupe
6. On prédit un groupe de tags à partir de la transformation LDA

In [23]:
def modelisation_LDA(clf, X_train2, X_test2, Y_train, Y_test, seuil_groupe_tags=0.8):
    debut0 = datetime.datetime.now()
    debut = datetime.datetime.now()
    #X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8, test_size=0.2)
    
    # Generation Encodage 
    genere_encodage(seuil=seuil_groupe_tags)
    
    # 0 - Encodage des Y selon le seuil
    Y_train_name_encode = encode_Y(Y_train)
    Y_test_name_encode = encode_Y(Y_test)

    # 1 - Transfo TF-IDF VECTORIZER
    nb_topics = Y_train_name_encode.nunique()
    print('nb_topics = ', nb_topics)
    
    # On entraine la modelisation LDA
    from sklearn.decomposition import LatentDirichletAllocation
    from sklearn.decomposition import NMF
    
    # On entraine la modelisation LDA
    lda = LatentDirichletAllocation(
        n_components=nb_topics, max_iter=5,
        learning_method='online', learning_offset=20.,random_state=0, 
        doc_topic_prior=0.1 , topic_word_prior=0.01
    )
    
    debut = datetime.datetime.now()
    X_train_lda_trained = lda.fit_transform(X_train2)
    fin = datetime.datetime.now()
    print('Tps de Traitement : X_train_lda_trained\t{}'.format(fin-debut))
    
    debut = datetime.datetime.now()
    # On tranforme la données de test
    X_test_lda_trained = lda.fit_transform(X_test2)
    fin = datetime.datetime.now()
    print('Tps de Traitement : X_test_lda_trained\t{}'.format(fin-debut))
    
    debut = datetime.datetime.now()
    print('\t[ ETAPE3 ]')

    ## Entrainement par le Classifier
    clf.fit(X_train_lda_trained, Y_train_name_encode)
    fin = datetime.datetime.now()
    print('Tps de Traitement : clf.fit\t{}'.format(fin-debut))
    debut = datetime.datetime.now()

    # On mesure le score
    score_clf = clf.score(X_test_lda_trained, Y_test_name_encode)
    print('\t-> score = ', score_clf)
    
    fin = datetime.datetime.now()
    print('Tps de Traitement : SCORE\t{}'.format(fin-debut))
    print('\nTps Total {}'.format(fin-debut0))
    
    return clf, clf.predict(X_test_lda_trained)

# Premier test via LDA avec une Régression Logistic
> Nous mettons un seuil de 0.8 (donc si les groupes pris en compte représentent 80 % des groupes possibles de tags, alors nous ne prenons pas les autres en compte

In [31]:
from sklearn.linear_model import LogisticRegression
clf1 = LogisticRegression(C=0.10, penalty='l2', tol=0.1,random_state=0, 
                          solver='lbfgs', multi_class='multinomial')

n_clf, y_pred_encoded = modelisation_LDA(
    clf1, 
    X_train2=X_train2, X_test2=X_test2, Y_train=Y_train, Y_test=Y_test,
    seuil_groupe_tags=0.8
)

mesure_performance(y_pred_encoded, Y_test)

nb_topics =  66
Tps de Traitement : X_train_lda_trained	0:00:45.115857
Tps de Traitement : X_test_lda_trained	0:00:10.327847
	[ ETAPE3 ]
Tps de Traitement : clf.fit	0:00:04.713412
	-> score =  0.4346
Tps de Traitement : SCORE	0:00:00.009389

Tps Total 0:01:19.247862
	PERFORMANCE MODELE : 56.8531  -- soit 0.9802 %


> La performance est la moyenne des tags qui sont bons pour chaque POST : ayant 58 tags, c'est la valeur maximale que nous pourrions obtenir

## Baissons le seuil pour le nombre de groupe à 70%

In [34]:
from sklearn.linear_model import LogisticRegression
clf1 = LogisticRegression(C=0.10, penalty='l2', tol=0.1,random_state=0, 
                          solver='lbfgs', multi_class='multinomial')

n_clf, y_pred_encoded = modelisation_LDA(
    clf1, 
    X_train2=X_train2, X_test2=X_test2, Y_train=Y_train, Y_test=Y_test,
    seuil_groupe_tags=0.7
)

mesure_performance(y_pred_encoded, Y_test)

nb_topics =  37
Tps de Traitement : X_train_lda_trained	0:00:38.504661
Tps de Traitement : X_test_lda_trained	0:00:07.448606
	[ ETAPE3 ]
Tps de Traitement : clf.fit	0:00:02.298049
	-> score =  0.5379
Tps de Traitement : SCORE	0:00:00.002227

Tps Total 0:01:07.152423
	PERFORMANCE MODELE : 56.853  -- soit 0.9802 %


# Maintenant augmentons ce seuil à 95%

In [35]:
from sklearn.linear_model import LogisticRegression
clf1 = LogisticRegression(C=0.10, penalty='l2', tol=0.1,random_state=0, 
                          solver='lbfgs', multi_class='multinomial')

n_clf, y_pred_encoded = modelisation_LDA(
    clf1, 
    X_train2=X_train2, X_test2=X_test2, Y_train=Y_train, Y_test=Y_test,
    seuil_groupe_tags=0.95
)

mesure_performance(y_pred_encoded, Y_test)

nb_topics =  368
Tps de Traitement : X_train_lda_trained	0:02:30.225838
Tps de Traitement : X_test_lda_trained	0:00:33.916603
	[ ETAPE3 ]
Tps de Traitement : clf.fit	0:00:48.512405
	-> score =  0.283
Tps de Traitement : SCORE	0:00:00.064979

Tps Total 0:04:12.146149
	PERFORMANCE MODELE : 56.8531  -- soit 0.9802 %


> La performance est très proche pour un temps d'entrainement plus long

> Faisons évoluer les hyperparamètres

In [36]:
from sklearn.linear_model import LogisticRegression
clf1 = LogisticRegression(C=10, penalty='l2', tol=0.01,random_state=0, 
                          solver='lbfgs', multi_class='multinomial')

n_clf, y_pred_encoded = modelisation_LDA(
    clf1, 
    X_train2=X_train2, X_test2=X_test2, Y_train=Y_train, Y_test=Y_test,
    seuil_groupe_tags=0.80
)

mesure_performance(y_pred_encoded, Y_test)

nb_topics =  66
Tps de Traitement : X_train_lda_trained	0:00:47.045701
Tps de Traitement : X_test_lda_trained	0:00:11.633321
	[ ETAPE3 ]
Tps de Traitement : clf.fit	0:00:06.051731
	-> score =  0.1868
Tps de Traitement : SCORE	0:00:00.008952

Tps Total 0:01:24.576898
	PERFORMANCE MODELE : 55.9436  -- soit 0.9645 %


> Le score baisse fortement avec les modifications des hyper paramètres

In [46]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer

vect_count = TfidfVectorizer(
    analyzer="word",
    min_df=100,
    max_df=0.2,
    tokenizer=tokenize,
    preprocessor=preProcess_keep_Code,
    stop_words=StopWord,        # Pas de stopWord car nous les cherchons
    ngram_range=(1, 6)
)

debut = datetime.datetime.now()
X_train2 = vect_count.fit_transform(X_train)
X_test2 = vect_count.transform(X_test)
fin = datetime.datetime.now()
print("Fin TfidfVectorizer : {}".format(fin-debut))

Fin TfidfVectorizer : 0:03:20.278255


# MNF

In [38]:
def modelisation_MNF(clf, X_train2, X_test2, Y_train, Y_test, seuil_groupe_tags=0.8):
    debut0 = datetime.datetime.now()
    debut = datetime.datetime.now()
    #X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8, test_size=0.2)
    
    # Generation Encodage 
    genere_encodage(seuil=seuil_groupe_tags)
    
    # 0 - Encodage des Y selon le seuil
    Y_train_name_encode = encode_Y(Y_train)
    Y_test_name_encode = encode_Y(Y_test)

    # 1 - Transfo TF-IDF VECTORIZER
    nb_topics = Y_train_name_encode.nunique()
    print('nb_topics = ', nb_topics)
    
    # On entraine la modelisation LDA
    from sklearn.decomposition import LatentDirichletAllocation
    from sklearn.decomposition import NMF

    # Run NMF
    mnf = NMF(n_components=nb_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd')
    
    debut = datetime.datetime.now()
    X_train_mnf_trained = mnf.fit_transform(X_train2)
    fin = datetime.datetime.now()
    print('Tps de Traitement : X_train_lda_trained\t{}'.format(fin-debut))
    
    debut = datetime.datetime.now()
    # On tranforme la données de test
    X_test_mnf_trained = mnf.fit_transform(X_test2)
    fin = datetime.datetime.now()
    print('Tps de Traitement : X_test_lda_trained\t{}'.format(fin-debut))
    
    debut = datetime.datetime.now()
    print('\t[ ETAPE3 ]')
   
    # On va standardiser les données
    #from sklearn.preprocessing import StandardScaler
    #std = StandardScaler()
    #std.fit(np.concatenate((X_train_mnf_trained,X_test_mnf_trained)))
    ########X_train_mnf_trained = std.transform(X_train_mnf_trained)
    ########X_test_mnf_trained = std.transform(X_test_mnf_trained)

    ## Entrainement par le Classifier
    clf.fit(X_train_mnf_trained, Y_train_name_encode)
    fin = datetime.datetime.now()
    print('Tps de Traitement : clf.fit\t{}'.format(fin-debut))
    debut = datetime.datetime.now()

    # On mesure le score
    score_clf = clf.score(X_test_mnf_trained, Y_test_name_encode)
    print('\t-> score = ', score_clf)
    
    fin = datetime.datetime.now()
    print('Tps de Traitement : SCORE\t{}'.format(fin-debut))
    print('\nTps Total {}'.format(fin-debut0))
    
    return clf, clf.predict(X_test_mnf_trained)

In [39]:
from sklearn.linear_model import LogisticRegression
clf1 = LogisticRegression(C=0.10, penalty='l2', tol=0.1,random_state=0, 
                          solver='lbfgs', multi_class='multinomial')

n_clf, y_pred_encoded = modelisation_MNF(
    clf1, 
    X_train2=X_train2, X_test2=X_test2, Y_train=Y_train, Y_test=Y_test,
    seuil_groupe_tags=0.8
)

mesure_performance(y_pred_encoded, Y_test)

nb_topics =  66
Tps de Traitement : X_train_lda_trained	0:00:31.409727
Tps de Traitement : X_test_lda_trained	0:00:03.903934
	[ ETAPE3 ]
Tps de Traitement : clf.fit	0:00:02.672424
	-> score =  0.4346
Tps de Traitement : SCORE	0:00:00.007964

Tps Total 0:00:56.935946
	PERFORMANCE MODELE : 56.8531  -- soit 0.9802 %


In [40]:
from sklearn.linear_model import LogisticRegression
clf1 = LogisticRegression(C=0.01, penalty='l2', tol=0.1,random_state=0, 
                          solver='lbfgs', multi_class='multinomial')

n_clf, y_pred_encoded = modelisation_MNF(
    clf1, 
    X_train2=X_train2, X_test2=X_test2, Y_train=Y_train, Y_test=Y_test,
    seuil_groupe_tags=0.8
)

mesure_performance(y_pred_encoded, Y_test)

nb_topics =  66
Tps de Traitement : X_train_lda_trained	0:00:33.253386
Tps de Traitement : X_test_lda_trained	0:00:04.248784
	[ ETAPE3 ]
Tps de Traitement : clf.fit	0:00:01.818632
	-> score =  0.4346
Tps de Traitement : SCORE	0:00:00.009006

Tps Total 0:00:58.013585
	PERFORMANCE MODELE : 56.8531  -- soit 0.9802 %


## Entrainement d'un modèle par RandomForest

In [41]:
from sklearn.ensemble import RandomForestClassifier
clf2 = RandomForestClassifier(n_estimators=1000, max_depth=10, random_state=0)

n_clf, y_pred_encoded = modelisation_MNF(
    clf2, 
    X_train2=X_train2, X_test2=X_test2, Y_train=Y_train, Y_test=Y_test,
    seuil_groupe_tags=0.8
)

mesure_performance(y_pred_encoded, Y_test)

/Users/seb/anaconda/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


nb_topics =  66
Tps de Traitement : X_train_lda_trained	0:00:31.802162
Tps de Traitement : X_test_lda_trained	0:00:03.949348
	[ ETAPE3 ]
Tps de Traitement : clf.fit	0:01:09.166269
	-> score =  0.4336
Tps de Traitement : SCORE	0:00:01.958972

Tps Total 0:02:05.695654
	PERFORMANCE MODELE : 56.85  -- soit 0.9802 %


# Travail sur Title

In [64]:
X_train_title = df1.loc[X_train.index]['Title']
X_test_title = df1.loc[X_test.index]['Title']

vect_tfidf = TfidfVectorizer(
    analyzer="word",
    min_df=100,
    max_df=0.2,
    tokenizer=tokenize,
    preprocessor=preProcess_keep_Code,
    stop_words=StopWord,        # Pas de stopWord car nous les cherchons
    ngram_range=(1, 6)
)

X_train_title2 = vect_tfidf.fit_transform(X_train_title)
X_test_title2 = vect_tfidf.transform(X_test_title)

In [70]:
from sklearn.linear_model import LogisticRegression

clf1 = LogisticRegression(C=0.01, penalty='l2', tol=0.1,random_state=0, 
                          solver='lbfgs', multi_class='multinomial')

n_clf, y_pred_encoded_title = modelisation_MNF(
    clf1, 
    X_train2=X_train_title2, X_test2=X_test_title2, Y_train=Y_train, Y_test=Y_test,
    seuil_groupe_tags=0.8
)

mesure_performance(y_pred_encoded_title, Y_test)

nb_topics =  66
Tps de Traitement : X_train_lda_trained	0:00:23.030431
Tps de Traitement : X_test_lda_trained	0:00:03.345411
	[ ETAPE3 ]
Tps de Traitement : clf.fit	0:00:01.424128
	-> score =  0.4346
Tps de Traitement : SCORE	0:00:00.008363

Tps Total 0:00:46.608810
	PERFORMANCE MODELE : 56.8531  -- soit 0.9802 %


In [71]:
from sklearn.linear_model import LogisticRegression
clf1 = LogisticRegression(C=0.01, penalty='l2', tol=0.1,random_state=0, 
                          solver='lbfgs', multi_class='multinomial')

n_clf, y_pred_encoded_body = modelisation_MNF(
    clf1, 
    X_train2=X_train2, X_test2=X_test2, Y_train=Y_train, Y_test=Y_test,
    seuil_groupe_tags=0.8
)

mesure_performance(y_pred_encoded_body, Y_test)

nb_topics =  66
Tps de Traitement : X_train_lda_trained	0:00:31.115794
Tps de Traitement : X_test_lda_trained	0:00:03.884413
	[ ETAPE3 ]
Tps de Traitement : clf.fit	0:00:01.511436
	-> score =  0.4346
Tps de Traitement : SCORE	0:00:00.007539

Tps Total 0:00:55.038852
	PERFORMANCE MODELE : 56.8531  -- soit 0.9802 %


## Si on applique un OR sur les Titre et Body

In [80]:
y_pred_encoded_title_OR_body = np.where(np.logical_or(y_pred_encoded_title, y_pred_encoded_body), 1, 0)
mesure_performance(y_pred_encoded_title_OR_body, Y_test)

	PERFORMANCE MODELE : 56.8531  -- soit 0.9802 %


> Pas de progression sur le résultat